<a href="https://colab.research.google.com/github/ns7900f/linux/blob/master/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pkill -f ngrok

In [ ]:
!pkill -9 ngrok


In [ ]:
!ngrok authtoken 2tjAvjV3Ji7yRumriu3SsOvhinz_bP17YkHirMjWeTw2AtfS


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install streamlit
# Install ngrok
!pip install pyngrok
!pip install pycaret
!pip install streamlit pycaret pandas openpyxl pyngrok

In [ ]:
!pip install pyngrok
#!ngrok authtoken 2tjAvjV3Ji7yRumriu3SsOvhinz_bP17YkHirMjWeTw2AtfS
from pyngrok import ngrok

import os
#os.environ["NGROK_AUTH_TOKEN"] = "2tjAvjV3Ji7yRumriu3SsOvhinz_bP17YkHirMjWeTw2AtfS"
ngrok.set_auth_token("2tjAvjV3Ji7yRumriu3SsOvhinz_bP17YkHirMjWeTw2AtfS")  # Replace with your token
tunnel = ngrok.connect(8502)
# Print the public URL of the tunnel
print("Public URL:", tunnel.public_url)

Public URL: https://24ac-34-68-63-71.ngrok-free.app


In [ ]:
#!ngrok http 8502


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & tail -n 10 /content/logs.txt

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import pycaret.classification as pycc
import pycaret.regression as pycr
from sklearn.preprocessing import LabelEncoder
from pycaret.classification import setup as setup_clf, compare_models as compare_clf, predict_model as predict_clf, create_model, pull, save_model
from pycaret.regression import setup as setup_reg, compare_models as compare_reg, predict_model as predict_reg, create_model as create_reg_model, pull as pull_reg, save_model as save_reg_model
from pycaret.classification import models as models_clf
from pycaret.regression import models as models_reg
from pyngrok import ngrok
import os

# Page setup
st.set_page_config(page_title="Purpel App", layout="wide")
st.sidebar.header("Upload Data")

# File uploader
uploaded_file = st.sidebar.file_uploader("Choose a file", type=["csv", "xlsx", "json"])
# Styling for upload button
st.markdown("""
    <style>
        .stFileUploader label {
            background-color: #8A2BE2 !important;
            color: white !important;
            padding: 10px !important;
            border-radius: 5px !important;
            font-weight: bold !important;
            text-align: center !important;
            display: block !important;
        }
    </style>
""", unsafe_allow_html=True)
df = None  # Ensure df is defined globally

if uploaded_file is not None:
    file_extension = uploaded_file.name.split(".")[-1]
    if file_extension == "csv":
        df = pd.read_csv(uploaded_file)
    elif file_extension in ["xls", "xlsx"]:
        df = pd.read_excel(uploaded_file)
    elif file_extension == "json":
        df = pd.read_json(uploaded_file)

    # Data Overview
    st.markdown("""<h2 style='color: #8A2BE2;'>Data Overview</h2>""", unsafe_allow_html=True)

    with st.expander("📌 Data Preview"):
        st.data_editor(df, height=300)

    with st.expander("📌 Show Data Types"):
        st.dataframe(pd.DataFrame(df.dtypes, columns=["Data Type"]))

    with st.expander("📌 Show Missing Values"):
        missing_values = df.isnull().sum().reset_index()
        missing_values.columns = ["Column", "Missing Values"]
        st.dataframe(missing_values.style.background_gradient(cmap="Reds"))

    # اختيار التارغيت في الشريط الجانبي بدون التأثير على عرض البيانات
    target = st.sidebar.selectbox("🎯 Choose Target Variable", [None] + list(df.columns), index=0)

    if target is None:
        st.sidebar.warning("⚠️ Please choose a target variable.")
    else:
        is_classification = df[target].dtype == 'object' or df[target].nunique() < 10
        st.sidebar.success("✔️ Classification Problem" if is_classification else "✔️ Regression Problem")

    # معالجة البيانات
    st.markdown("""<h2 style='color: #8A2BE2;'>🛠️ Data Processing</h2>""", unsafe_allow_html=True)
    processing_option = st.radio("Choose Processing Method", ["None", "Drop Columns", "Handle Missing Values", "Label Encoding", "One-Hot Encoding"])

    if processing_option == "Drop Columns":
        drop_cols = st.multiselect("🗑️ Select Columns to Drop", df.columns)
        df = df.drop(columns=drop_cols)
        with st.expander("📌 Data After Dropping Columns"):
            st.write(df)

    le = LabelEncoder()

    if processing_option == "Label Encoding":
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].fillna("missing")
            df[col] = le.fit_transform(df[col].astype(str))
        with st.expander("📌 Data After Label Encoding"):
            st.write(df)

    if processing_option == "Handle Missing Values":
        handling_method = st.radio("🧩 Choose Missing Value Handling Method", ["Mean", "Median", "Mode", "Drop Rows"])

        if handling_method in ["Mean", "Median"]:
            for col in df.select_dtypes(include=['object']).columns:
                df[col] = df[col].fillna("missing")
                df[col] = le.fit_transform(df[col].astype(str))

        if handling_method == "Mean":
            df.fillna(df.mean(), inplace=True)
        elif handling_method == "Median":
            df.fillna(df.median(), inplace=True)
        elif handling_method == "Mode":
            df.fillna(df.mode().iloc[0], inplace=True)
        elif handling_method == "Drop Rows":
            df.dropna(inplace=True)

        with st.expander("📌 Data After Handling Missing Values"):
            st.write(df)

    if processing_option == "One-Hot Encoding":
        df = pd.get_dummies(df)
        with st.expander("📌 Data After One-Hot Encoding"):
            st.write(df)

    # Model Training
    if target is not None:
        if is_classification:
            exp = setup_clf(df, target=target, verbose=False)
            available_models = list(models_clf().index)
        else:
            exp = setup_reg(df, target=target, verbose=False)
            available_models = list(models_reg().index)

        selected_models = st.sidebar.multiselect("📌 Select models to train", available_models)

        if st.sidebar.button("Train Models"):
             if selected_models:
                best_model = compare_clf(include=selected_models) if is_classification else compare_reg(include=selected_models)
                st.write("### 🏆 Best Model Details")
                st.write(best_model)

                model_params = pull() if is_classification else pull_reg()
                st.write("📊 Model Hyperparameters and Performance")
                st.dataframe(model_params)

                predictions = predict_clf(best_model, data=df) if is_classification else predict_reg(best_model, data=df)

                st.write("📊 Model Comparison Report")
                st.dataframe(pull() if is_classification else pull_reg())

                st.write("✅ Best Model Created:")
                model = create_model(best_model) if is_classification else create_reg_model(best_model)
                st.dataframe(pull() if is_classification else pull_reg())

                st.write("### 🔍 Predictions")
                st.write(predictions)

                if st.sidebar.button("💾 Save Best Model"):
                  os.makedirs("models", exist_ok=True)
                  if is_classification:
                      save_model(model, "models/best_model")
                  else:
                      save_reg_model(model, "models/best_model")
                  st.sidebar.success("✅ Model saved successfully!")

                if st.sidebar.button("🌍 Deploy via Ngrok"):
                    public_url = ngrok.connect(8501)
                    st.sidebar.success(f"📢 App available at: {public_url}")
                    st.sidebar.write(f"🌍 **Click here to access:** [Open App]({public_url})")

        else:
                st.sidebar.error("❌ Please select at least one model.")


#os.environ["NGROK_AUTH_TOKEN"] = "2tjAvjV3Ji7yRumriu3SsOvhinz_bP17YkHirMjWeTw2AtfS"
#ngrok.set_auth_token("2tjAvjV3Ji7yRumriu3SsOvhinz_bP17YkHirMjWeTw2AtfS")  # Replace with your token
#tunnel = ngrok.connect(8501)
# Print the public URL of the tunnel
#print("Public URL:", tunnel.public_url)



Overwriting app.py


In [ ]:
!streamlit run app.py --server.port 8502




2025-03-02 20:09:53.711 Port 8502 is already in use
